# Analysis of NEISS Product Injury Database

### Import the libraries

In [1]:
import os, time
import pandas as pd
import sqlite3

### Process raw excel files into a database

In [84]:
%%time
fact_directory = 'NEISS_All_Data/Fact Tables/'

dfs = []

# This code iterates through the list of documents in the directory 
# and loads them into a list of dataframes.
''' 
for filename in os.listdir(fact_directory): 
    filename_l = filename.lower() 
    if filename_l.endswith('xlsx') and filename_l.startswith('neiss'): 
        filepath = os.path.join(fact_directory, filename) 
        df = pd.read_excel(filepath)
        dfs.append(df)
data_load_finish = time.time() - start_time
print(data_load_finish)

# Concatenate the list of dataframes into one large dataframe
all_neiss = pd.concat(dfs, ignore_index=True) 
'''

# Loads the files individually and processes them into the database
df = pd.read_excel('NEISS_All_Data/Fact Tables/NEISS_2003.XLSX')
df.to_sql('all_neiss', neiss_conn, if_exists = 'append', index=True)

CPU times: user 1min 39s, sys: 5.12 s, total: 1min 44s
Wall time: 1min 47s


347375

Sample times for individual files Macbook air 2019 1.6 Ghz Dual-Core Intel:
<br>2:16
<br>1:57
<br>2:11
<br>2:11
<br>2:44
<br>2:37
<br>2:09
2:15
2:03
(after plugging it in)
1:55
1:51
1:44

In [61]:
data_load_finish / 60

3.280905465284983

In [24]:
all_neiss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1523625 entries, 0 to 1523624
Data columns (total 27 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   CPSC_Case_Number   1523625 non-null  int64         
 1   Treatment_Date     1523625 non-null  datetime64[ns]
 2   Age                1523625 non-null  int64         
 3   Sex                1523625 non-null  int64         
 4   Race               1523625 non-null  int64         
 5   Other_Race         131286 non-null   object        
 6   Hispanic           0 non-null        float64       
 7   Body_Part          1523625 non-null  int64         
 8   Diagnosis          1523625 non-null  int64         
 9   Diagoth            105974 non-null   object        
 10  Body_Part_2        0 non-null        float64       
 11  Diagnosis_2        0 non-null        float64       
 12  Diag2oth           0 non-null        float64       
 13  Disposition        1523625 

### Load the dimension tables

In [38]:
dim_directory = 'NEISS_All_Data/Dimension Tables/'
dataframes = {}

for filename in os.listdir(dim_directory):
    if filename.endswith('.xlsx'):
        filepath = os.path.join(dim_directory, filename)
        dataframe_name = filename.split('.')[0]  # Use the filename as the dataframe name
        dataframes[dataframe_name] = pd.read_excel(filepath)

### Load the dimension tables into the database

In [40]:
for dataframe_name, dataframe in dataframes.items():
    dataframe.to_sql(dataframe_name, neiss_conn, if_exists='replace', index=False)

In [3]:
neiss_conn = sqlite3.connect('neiss.db')
neiss_cursor = neiss_conn.cursor()

In [28]:
all_neiss.to_sql('all_neiss', neiss_conn, if_exists='replace', index=True)

1523625

In [81]:
%%time

view = pd.read_sql(
    
'''

SELECT DISTINCT SUBSTRING(Treatment_Date, 1, 4) AS first_four_digits
FROM all_neiss
ORDER BY first_four_digits Desc


'''
,neiss_conn)
view

,first_four_digits
0,2022
1,2021
2,2020
3,2019
4,2018
5,2017
6,2016
7,2015
8,2014
9,2013


## View all of the tables in the database

In [41]:
view = pd.read_sql(
'''

SELECT name FROM sqlite_master WHERE type='table'

'''
,neiss_conn)
view

,name
0,all_neiss
1,AgeLTwoDim
2,DispositionDim
3,RaceDim
4,FireDim
5,LocationDim
6,GenderDim
7,HispanicDim
8,BdypartDim
9,DiagnosisDim


In [85]:
%%time

view = pd.read_sql(
'''

SELECT * FROM all_neiss
WHERE Treatment_Date LIKE '%2012%'

'''
,neiss_conn)
view

CPU times: user 7.81 s, sys: 3.41 s, total: 11.2 s
Wall time: 31.5 s


,index,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Hispanic,Body_Part,Diagnosis,...,Drug,Product_1,Product_2,Product_3,Narrative,Stratum,PSU,Weight,Other_Diagnosis,Other_Diagnosis_2
0,1129242,120103891,2012-01-02 00:00:00,54,2,1,None,None,75,59,...,None,4057,0,0,"54YOF, GOT UP AT HOME TO GO TO BATHROOM, FELL ...",S,71,69.9872,None,None
1,1129243,120103905,2012-01-01 00:00:00,35,2,2,None,None,85,65,...,None,1143,0,0,"35YOF, SMOKE INHALATION & SHOULDER PAIN FROM C...",S,71,69.9872,None,None
2,1129244,120103908,2012-01-02 00:00:00,18,1,2,None,None,30,55,...,None,1205,0,0,"18YOM, HURT SHOULDER WHILE PLAYING BASKETBALL ...",S,71,69.9872,None,None
3,1129245,120103909,2012-01-02 00:00:00,17,1,2,None,None,93,64,...,None,1211,0,0,"17YOM, STEPPED IN HOLE WHILE PLAYING FOOTBALL....",S,71,69.9872,None,None
4,1129246,120103911,2012-01-02 00:00:00,49,1,2,None,None,37,64,...,None,1807,0,0,"49YOM, SLIPPED IN WATER ON FLOOR IN JAIL. DX: ...",S,71,69.9872,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394378,1523620,130347137,2012-02-11 00:00:00,12,1,1,None,None,30,53,...,None,1205,0,0,12YOM FELL X2 ON SHOULDER PLAYING BASKETBALL/S...,S,66,69.9872,None,None
394379,1523621,130347138,2012-02-11 00:00:00,2,1,1,None,None,31,53,...,None,1506,1842,0,2YOM MOVED BABY GATE & FELL DOWN 15 WOODEN STE...,S,66,69.9872,None,None
394380,1523622,130347139,2012-02-11 00:00:00,2,2,1,None,None,92,71,...,None,1807,0,0,2YOF HAD TEMPER TANTRUM & THREW SELF ON VINYL ...,S,66,69.9872,None,None
394381,1523623,130347141,2012-02-11 00:00:00,40,1,1,None,None,31,71,...,None,611,0,0,40YOM GETTING OUT OF SHOWER FELT POP IN BACK/E...,S,66,69.9872,None,None


In [4]:
%%time

view = pd.read_sql(
'''

SELECT * FROM all_neiss

'''
,neiss_conn)
view

CPU times: user 58.5 s, sys: 29.8 s, total: 1min 28s
Wall time: 1min 28s


,index,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Hispanic,Body_Part,Diagnosis,...,Drug,Product_1,Product_2,Product_3,Narrative,Stratum,PSU,Weight,Other_Diagnosis,Other_Diagnosis_2
0,0,90101432,2009-01-01 00:00:00,5,1,3,HISPANIC,NaN,89,64,...,NaN,1807,0,0,5 YOM ROLLING ON FLOOR DOING A SOMERSAULT AND ...,V,61,15.3491,None,None
1,1,90101434,2009-01-01 00:00:00,51,1,1,None,NaN,77,53,...,NaN,899,0,0,"51 YOM C/O PAIN AND IRRITATION TO RIGHT EYE, H...",V,61,15.3491,None,None
2,2,90101435,2009-01-01 00:00:00,2,2,1,None,NaN,76,59,...,NaN,4057,0,0,2 YOF WAS RUNNING THROUGH HOUSE AND FELL INTO ...,V,61,15.3491,None,None
3,3,90101436,2009-01-01 00:00:00,20,1,1,None,NaN,93,53,...,NaN,1884,0,0,20 YOM PUNCHED AND KICKED A WALL D/T DRINKING ...,V,61,15.3491,None,None
4,4,90101437,2009-01-01 00:00:00,20,1,1,None,NaN,34,57,...,NaN,3283,0,0,20 YOM FELL SNOW SKIING SUSTAINING A FRACTURED...,V,61,15.3491,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7316556,347370,40605786,2003-05-31 00:00:00,2,2,2,None,NaN,76,53,...,NaN,827,0,0,PATIENT PULLED HAMMER DOWN AND HIT EYE; FACIAL...,C,8,6.1043,None,None
7316557,347371,40605787,2003-05-31 00:00:00,206,2,1,None,NaN,30,53,...,NaN,1807,4057,0,"BABYSITTER DROPPED PATIENT ONTO FLOOR, HIT LEF...",C,8,6.1043,None,None
7316558,347372,40605789,2003-05-31 00:00:00,2,2,2,None,NaN,76,59,...,NaN,679,4057,0,PATIENT FELL OFF COUCH HITTING HEAD ON WOODEN ...,C,8,6.1043,None,None
7316559,347373,40605791,2003-05-31 00:00:00,11,1,2,None,NaN,76,59,...,NaN,1205,1871,0,"PATIENT PLAYING BASKETBALL, FELL INTO METAL FE...",C,8,6.1043,None,None


Load takes 3:59 on macbook.
1:28 on System76 running 2 cores. 